In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# 1) Load data
# car_df = pd.read_csv('car_fuel_efficiency.csv', encoding='ascii')
cars_df = pd.read_csv('car_fuel_efficiency.csv')
print(cars_df.head())

   engine_displacement  num_cylinders  horsepower  vehicle_weight  \
0                  170            3.0       159.0     3413.433759   
1                  130            5.0        97.0     3149.664934   
2                  170            NaN        78.0     3079.038997   
3                  220            4.0         NaN     2542.392402   
4                  210            1.0       140.0     3460.870990   

   acceleration  model_year  origin fuel_type         drivetrain  num_doors  \
0          17.7        2003  Europe  Gasoline    All-wheel drive        0.0   
1          17.8        2007     USA  Gasoline  Front-wheel drive        0.0   
2          15.1        2018  Europe  Gasoline  Front-wheel drive        0.0   
3          20.2        2009     USA    Diesel    All-wheel drive        2.0   
4          14.4        2009  Europe  Gasoline    All-wheel drive        2.0   

   fuel_efficiency_mpg  
0            13.231729  
1            13.688217  
2            14.246341  
3         

In [ ]:
# Keep only needed columns
cols = ['engine_displacement','horsepower','vehicle_weight','model_year','fuel_efficiency_mpg']
df = cars_df[cols].copy()
print(df.head())

   engine_displacement  horsepower  vehicle_weight  model_year  \
0                  170       159.0     3413.433759        2003   
1                  130        97.0     3149.664934        2007   
2                  170        78.0     3079.038997        2018   
3                  220         NaN     2542.392402        2009   
4                  210       140.0     3460.870990        2009   

   fuel_efficiency_mpg  
0            13.231729  
1            13.688217  
2            14.246341  
3            16.912736  
4            12.488369  


In [ ]:
# EDA: check long tail via simple skewness
fe = df['fuel_efficiency_mpg']
skew_val = fe.skew()
print(skew_val)

-0.012062219273507929


In [ ]:
# Q1: column with missing values among selected
na_counts = df.isna().sum()
col_with_na = na_counts[na_counts > 0].index.tolist()
print(col_with_na)

['horsepower']


In [ ]:
# Q2: median of horsepower
hp_median = np.nanmedian(df['horsepower'].values)
print(hp_median)

149.0


In [ ]:
# Prepare split function
def split_60_20_20(data, seed):
    d_shuf = data.sample(frac=1.0, random_state=seed).reset_index(drop=True)
    n = len(d_shuf)
    n_train = int(0.6*n)
    n_val = int(0.2*n)
    df_train = d_shuf.iloc[:n_train]
    df_val = d_shuf.iloc[n_train:n_train+n_val]
    df_test = d_shuf.iloc[n_train+n_val:]
    return df_train, df_val, df_test

In [ ]:
# Helper to train OLS closed-form (no reg)
def train_predict_ols(X_train, y_train, X_val):
    Xb = np.column_stack([np.ones(len(X_train)), X_train])
    Xvb = np.column_stack([np.ones(len(X_val)), X_val])
    w = np.linalg.pinv(Xb.T.dot(Xb)).dot(Xb.T).dot(y_train)
    y_pred = Xvb.dot(w)
    return y_pred

In [ ]:
# Helper ridge (reg r)
def train_predict_ridge(X_train, y_train, X_val, r):
    Xb = np.column_stack([np.ones(len(X_train)), X_train])
    Xvb = np.column_stack([np.ones(len(X_val)), X_val])
    I = np.eye(Xb.shape[1])
    I[0,0] = 0.0
    w = np.linalg.pinv(Xb.T.dot(Xb) + r*I).dot(Xb.T).dot(y_train)
    y_pred = Xvb.dot(w)
    return y_pred

In [ ]:
# Split with seed 42
train_df, val_df, test_df = split_60_20_20(df, 42)
print(len(train_df))
print(len(val_df))
print(len(test_df))

5822
1940
1942


In [ ]:
# Features/target
feat_cols = ['engine_displacement','horsepower','vehicle_weight','model_year']

In [ ]:
# Q3: missing handling for the column(s) with NA (apply just to horsepower if that's the one)
# Option A: fill with 0
train_A = train_df.copy()
val_A = val_df.copy()
for c in feat_cols:
    if train_A[c].isna().any() or val_A[c].isna().any():
        train_A[c] = train_A[c].fillna(0)
        val_A[c] = val_A[c].fillna(0)
XA = train_A[feat_cols].values
ya = train_A['fuel_efficiency_mpg'].values
XvA = val_A[feat_cols].values
predA = train_predict_ols(XA, ya, XvA)
rmse_A = np.sqrt(mean_squared_error(val_A['fuel_efficiency_mpg'].values, predA))
print(rmse_A)

0.5171866959591351


In [ ]:
# Option B: fill with mean (compute on train only)
train_B = train_df.copy()
val_B = val_df.copy()
for c in feat_cols:
    if train_B[c].isna().any() or val_B[c].isna().any():
        m = train_B[c].mean()
        train_B[c] = train_B[c].fillna(m)
        val_B[c] = val_B[c].fillna(m)
XB = train_B[feat_cols].values
yb = train_B['fuel_efficiency_mpg'].values
XvB = val_B[feat_cols].values
predB = train_predict_ols(XB, yb, XvB)
rmse_B = np.sqrt(mean_squared_error(val_B['fuel_efficiency_mpg'].values, predB))
print(rmse_B)

0.46353660135009683


In [ ]:
# Q4: ridge with r list, fill NAs with 0 as instructed
r_list = [0, 0.01, 0.1, 1, 5, 10, 100]
train_Z = train_df.copy()
val_Z = val_df.copy()
for c in feat_cols:
    train_Z[c] = train_Z[c].fillna(0)
    val_Z[c] = val_Z[c].fillna(0)
XZ_tr = train_Z[feat_cols].values
yZ_tr = train_Z['fuel_efficiency_mpg'].values
XZ_val = val_Z[feat_cols].values
rmse_by_r = []
for r in r_list:
    yp = train_predict_ridge(XZ_tr, yZ_tr, XZ_val, r)
    rmse = np.sqrt(mean_squared_error(val_Z['fuel_efficiency_mpg'].values, yp))
    rmse_by_r.append((r, rmse))
print(rmse_by_r)

[(0, np.float64(0.5171866959591351)), (0.01, np.float64(0.5171866960679854)), (0.1, np.float64(0.5171866960525745)), (1, np.float64(0.5171866967546045)), (5, np.float64(0.5171866998844799)), (10, np.float64(0.5171867036845023)), (100, np.float64(0.5171867731528449))]


In [ ]:
# Q5: seeds 0-9, OLS, fillna 0, collect val RMSE and std
val_rmses = []
for seed in range(10):
    tr, va, te = split_60_20_20(df, seed)
    for c in feat_cols:
        tr[c] = tr[c].fillna(0)
        va[c] = va[c].fillna(0)
    Xtr = tr[feat_cols].values
    ytr = tr['fuel_efficiency_mpg'].values
    Xva = va[feat_cols].values
    yva = va['fuel_efficiency_mpg'].values
    ypred = train_predict_ols(Xtr, ytr, Xva)
    rmse = np.sqrt(mean_squared_error(yva, ypred))
    val_rmses.append(rmse)
std_scores = float(np.std(val_rmses))
print(val_rmses)
print(std_scores)

[np.float64(0.5210993169564573), np.float64(0.5218422384203811), np.float64(0.5230384518938906), np.float64(0.5161215338335037), np.float64(0.5111869531897599), np.float64(0.5286833856280085), np.float64(0.5322418918226826), np.float64(0.5095258234083074), np.float64(0.5149083790071103), np.float64(0.5131330922229631)]
0.0071263191085892555


In [ ]:
# Q6: Split with seed 9, combine train+val, fillna(0), train ridge with r=0.001, evaluate RMSE on test
# Steps:
# - Load CSV
# - Select required columns
# - Train/val/test split with seed 9 (60/20/20)
# - Combine train+val for training
# - Impute 0 for missing
# - Standardize features (fit on train+val only)
# - Ridge with alpha=0.001 (r)
# - Predict on test and compute RMSE

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Load data
df = pd.read_csv('car_fuel_efficiency.csv', encoding='ascii')
cols = ['engine_displacement','horsepower','vehicle_weight','model_year','fuel_efficiency_mpg']
df2 = df[cols].copy()

# Split with seed 9
train_val_df, test_df = train_test_split(df2, test_size=0.2, random_state=9, shuffle=True)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=9, shuffle=True)

# Combine train+val
comb_df = pd.concat([train_df, val_df], axis=0)

# Separate features/target
feat_cols = ['engine_displacement','horsepower','vehicle_weight','model_year']
X_train = comb_df[feat_cols].fillna(0).values
y_train = comb_df['fuel_efficiency_mpg'].values
X_test = test_df[feat_cols].fillna(0).values
y_test = test_df['fuel_efficiency_mpg'].values

# Scale features
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# Ridge with r=0.001
model = Ridge(alpha=0.001, random_state=9)
model.fit(X_train_s, y_train)

# Predict and RMSE
pred = model.predict(X_test_s)
rmse = np.sqrt(mean_squared_error(y_test, pred))

print(rmse)
print(len(comb_df))
print(len(test_df))

0.5210336117018122
7763
1941
